# Neural Networks Architecture for Decoding EEG MI Data using Spectrogram Representations

## Preparation

In case that gumpy is not installed as a module, we need to specify the path to ``gumpy``. In addition, we wish to configure jupyter notebooks and any backend properly. Note that it may take some time for ``gumpy`` to load due to the number of dependencies

In [1]:
from __future__ import print_function
import os; os.environ["THEANO_FLAGS"] = "device=gpu0"
import os.path
from datetime import datetime
import sys
sys.path.append('./gumpy')

import gumpy
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
%matplotlib inline


To use the models provided by `gumpy-deeplearning`, we have to set the path to the models directory and import it. If you installed `gumpy-deeplearning` as a module, this step may not be required.

## Utility functions

The examples for ``gumpy-deeplearning`` ship with a few tiny helper functions. For instance, there's one that tells you the versions of the currently installed keras and kapre. ``keras`` is required in ``gumpy-deeplearning``, while ``kapre`` 
can be used to compute spectrograms.

In addition, the utility functions contain a method ``load_preprocess_data`` to load and preprocess data. Its usage will be shown further below

In [2]:
import utils
utils.print_version_info()

Using TensorFlow backend.
C:\ProgramData\Anaconda3\envs\ML_Extra_Credit\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\ML_Extra_Credit\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\ML_Extra_Credit\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\An

2020/2/12
Keras version: 2.2.4
Keras backend: tensorflow: 1.13.1
Keras image dim ordering: tf
Kapre version: 0.1.4


## Setup parameters for the model and data
Before we jump into the processing, we first wish to specify some parameters (e.g. frequencies) that we know from the data.

In [3]:
DEBUG = True
CLASS_COUNT = 2
DROPOUT = 0.2   # dropout rate in float

# parameters for filtering data
FS = 250
LOWCUT = 2
HIGHCUT = 60
ANTI_DRIFT = 0.5
CUTOFF = 50.0 # freq to be removed from signal (Hz) for notch filter
Q = 30.0  # quality factor for notch filter 
W0 = CUTOFF/(FS/2)
AXIS = 0

#set random seed
SEED = 42
KFOLD = 5
NumbItr=10

## Load raw data

Before training and testing a model, we need some data. The following code shows how to load a dataset using ``gumpy``.

In [4]:
# specify the location of the GrazB datasets
data_dir = './data/Graz'
subject = 'B01'

# initialize the data-structure, but do _not_ load the data yet
grazb_data = gumpy.data.GrazB(data_dir, subject)

# now that the dataset is setup, we can load the data. This will be handled from within the utils function, 
# which will first load the data and subsequently filter it using a notch and a bandpass filter.
# the utility function will then return the training data.
x_train, y_train = utils.load_preprocess_data(grazb_data, True, LOWCUT, HIGHCUT, W0, Q, ANTI_DRIFT, CLASS_COUNT, CUTOFF, AXIS, FS)


Band-pass filtering the data in frequency range from 2.0 Hz to 60.0 Hz... 
Data loaded and processed successfully!


## Augment data

In [5]:
x_augmented, y_augmented = gumpy.signal.sliding_window(data = x_train[:,:,:],
                                                          labels = y_train[:,:],
                                                          window_sz = 4 * FS,
                                                          n_hop = FS // 10,
                                                          n_start = FS * 1)
x_subject = x_augmented
y_subject = y_augmented
x_subject = np.rollaxis(x_subject, 2, 1)

# CNN model

In [6]:
#from .model import KerasModel
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import BatchNormalization, Dropout, Conv2D, MaxPooling2D
from keras.models import load_model
import kapre
from kapre.utils import Normalization2D
from kapre.time_frequency import Spectrogram

def CNN_model(input_shape, dropout=0.5, print_summary=False):

        # basis of the CNN_STFT is a Sequential network
        model = Sequential()

        # spectrogram creation using STFT
        model.add(Spectrogram(n_dft = 128, n_hop = 16, input_shape = input_shape,
                  return_decibel_spectrogram = False, power_spectrogram = 2.0,
                  trainable_kernel = False, name = 'static_stft'))
        model.add(Normalization2D(str_axis = 'freq'))

        # Conv Block 1
        model.add(Conv2D(filters = 24, kernel_size = (12, 12),
                         strides = (1, 1), name = 'conv1',
                         border_mode = 'same'))
        model.add(BatchNormalization(axis = 1))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size = (2, 2), strides = (2,2), padding = 'valid',
                               data_format = 'channels_last'))

        # Conv Block 2
        model.add(Conv2D(filters = 48, kernel_size = (8, 8),
                         name = 'conv2', border_mode = 'same'))
        model.add(BatchNormalization(axis = 1))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid',
                               data_format = 'channels_last'))

        # Conv Block 3
        model.add(Conv2D(filters = 96, kernel_size = (4, 4),
                         name = 'conv3', border_mode = 'same'))
        model.add(BatchNormalization(axis = 1))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size = (2, 2), strides = (2,2),
                               padding = 'valid',
                               data_format = 'channels_last'))
        model.add(Dropout(dropout))

        # classificator
        model.add(Flatten())
        model.add(Dense(2))  # two classes only
        model.add(Activation('softmax'))

        if print_summary:
            print(model.summary())

        # compile the model
        model.compile(loss = 'categorical_crossentropy',
                      optimizer = 'adam',
                      metrics = ['accuracy'])

        # assign model and return
        
        
        return model


In [7]:
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import ModelCheckpoint
# define KFOLD-fold cross validation test harness
kfold = StratifiedKFold(n_splits = KFOLD, shuffle = True, random_state = SEED)
cvscores = []
ii = 1
for train, test in kfold.split(x_subject, y_subject[:, 0]):
    print('Run ' + str(ii) + '...')
    # create callbacks
    model_name_str = 'GRAZ_CNN_STFT_3layer_' + \
                     '_run_' + str(ii)
    
    checkpoint = ModelCheckpoint(model_name_str, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    # Fit the model
    # initialize and create the model
    model = CNN_model(x_subject.shape[1:], dropout = DROPOUT, print_summary = False)
    
    # fit model. If you specify monitor=True, then the model will create callbacks
    # and write its state to a HDF5 file
    model.fit(x_subject[train], y_subject[train],
              epochs = NumbItr, 
              batch_size = 256, 
              verbose = 0, 
              validation_split = 0.1, callbacks = callbacks_list)

    # evaluate the model
    print('Evaluating model on test set...')
    scores = model.evaluate(x_subject[test], y_subject[test], verbose = 0)
    print("Result on test set: %s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
    cvscores.append(scores[1] * 100)
    ii += 1
    
# print some evaluation statistics and write results to file
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
cv_all_subjects = np.asarray(cvscores)
print('Saving CV values to file....')
np.savetxt('GRAZ_CV_' + 'CNN_STFT_3layer_' + str(DROPOUT) + 'do'+'.csv', 
            cv_all_subjects, delimiter = ',', fmt = '%2.4f')
print('CV values successfully saved!\n')

Run 1...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.

Epoch 00001: val_acc improved from -inf to 0.76368, saving model to GRAZ_CNN_STFT_3layer__run_1

Epoch 00002: val_acc did not improve from 0.76368

Epoch 00003: val_acc did not improve from 0.76368

Epoch 00004: val_acc improved from 0.76368 to 0.84826, saving model to GRAZ_CNN_STFT_3layer__run_1

Epoch 00005: val_acc did not improve from 0.84826

Epoch 00006: val_acc did not improve from 0.84826

Epoch 00007: val_acc did not improve from 0.84826

Epoch 00008: val_acc did not improve from 0.84826

Epoch 00009: val_acc did not improve from 0.84826

Epoch 00010: val_acc did not improve from 0.84826
Evaluating model on test set...
Result on test set: acc: 87.41%
Run 2...

Epoch 00001: val_acc improved from -inf to 0.87313, saving model to GRAZ_CNN_

C:\ProgramData\Anaconda3\envs\ML_Extra_Credit\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=24, kernel_size=(12, 12), strides=(1, 1), name="conv1", padding="same")`
C:\ProgramData\Anaconda3\envs\ML_Extra_Credit\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=48, kernel_size=(8, 8), name="conv2", padding="same")`
C:\ProgramData\Anaconda3\envs\ML_Extra_Credit\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=96, kernel_size=(4, 4), name="conv3", padding="same")`


## Load the trained model 

In [8]:
model.save('CNN_STFTmonitoring.h5')  # creates a HDF5 file 'my_model.h5'
model2 = load_model('CNN_STFTmonitoring.h5', 
                                 custom_objects={'Spectrogram': kapre.time_frequency.Spectrogram, 
                                                 'Normalization2D': kapre.utils.Normalization2D})

int_axis=1 passed but is ignored, str_axis is used instead.


# LSTM

In [9]:
from keras.layers import SimpleRNN, Dense, LSTM as _LSTM
def LSTM_model(input_shape, num_hidden_neurons=128,
                      num_layers=1, dropout=0.2, recurrent_dropout=0.2,
                      print_summary=False):
    model = Sequential()
    if num_layers > 1:
        for i in range(1, num_layers, 1):
            model.add(_LSTM(num_hidden_neurons, input_shape=input_shape,
                            return_sequences=True, dropout=dropout, recurrent_dropout=recurrent_dropout))
        model.add(_LSTM(num_hidden_neurons))
    else:
        model.add(_LSTM(num_hidden_neurons, input_shape=input_shape, dropout=dropout,
                        recurrent_dropout=recurrent_dropout))
    model.add(Dense(2, activation='softmax'))

    if print_summary:
        print(model.summary())

        # compile the model
    model.compile(loss='categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])

        # assign and return
    
    return model

In [10]:
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import ModelCheckpoint
# define KFOLD-fold cross validation test harness
kfold = StratifiedKFold(n_splits = KFOLD, shuffle = True, random_state = SEED)
cvscores = []
ii = 1
for train, test in kfold.split(x_subject, y_subject[:, 0]):
    print('Run ' + str(ii) + '...')
    # create callbacks
    model_name_str = 'GRAZ_LSTM_' + \
                     '_run_' + str(ii)
    
    checkpoint = ModelCheckpoint(model_name_str, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    # Fit the model
    # initialize and create the model
    model = LSTM_model(x_subject.shape[1:], dropout = DROPOUT, print_summary = False)
    
    # fit model. If you specify monitor=True, then the model will create callbacks
    # and write its state to a HDF5 file
    model.fit(x_subject[train], y_subject[train],
              epochs = NumbItr, 
              batch_size = 256, 
              verbose = 1, 
              validation_split = 0.1, callbacks = callbacks_list)

    # evaluate the model
    print('Evaluating model on test set...')
    scores = model.evaluate(x_subject[test], y_subject[test], verbose = 0)
    print("Result on test set: %s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
    cvscores.append(scores[1] * 100)
    ii += 1
    
# print some evaluation statistics and write results to file
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
cv_all_subjects = np.asarray(cvscores)
print('Saving CV values to file....')
np.savetxt('GRAZ_CV_' + 'LSTM_' + str(DROPOUT) + 'do'+'.csv', 
            cv_all_subjects, delimiter = ',', fmt = '%2.4f')
print('CV values successfully saved!\n')

Run 1...
Train on 7231 samples, validate on 804 samples
Epoch 1/10
7231/7231 [==============================] - 3s 483us/step - loss: 0.7361 - acc: 0.5183 - val_loss: 0.8449 - val_acc: 0.3619

## Load the trained model 

In [11]:
model.save('LSTMmonitoring.h5')  # creates a HDF5 file 'my_model.h5'
model2 = load_model('LSTMmonitoring.h5', 
                                 custom_objects={'Spectrogram': kapre.time_frequency.Spectrogram, 
                                                 'Normalization2D': kapre.utils.Normalization2D})

# MLP model

In [12]:
#from .model import KerasModel
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import BatchNormalization, Dropout, Conv2D, MaxPooling2D

import kapre
from kapre.utils import Normalization2D
from kapre.time_frequency import Spectrogram

def MLP_model(input_shape, dropout=0.5, print_summary=False):

        # basis of the CNN_STFT is a Sequential network
        model = Sequential()

        # spectrogram creation using STFT
        model.add(Spectrogram(n_dft = 128, n_hop = 16, input_shape = input_shape,
                  return_decibel_spectrogram = False, power_spectrogram = 2.0,
                  trainable_kernel = False, name = 'static_stft'))
        model.add(Normalization2D(str_axis = 'freq'))
        model.add(Flatten())  
        model.add(Dense(100, activation='relu', input_shape=(784,)))
        model.add(Dropout(0.2))
        model.add(Dense(100, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(2))  # two classes only
        model.add(Activation('softmax'))

        if print_summary:
            print(model.summary())

        # compile the model
        model.compile(loss = 'categorical_crossentropy',
                      optimizer = 'adam',
                      metrics = ['accuracy'])

        # assign model and return
        
        
        return model


In [13]:
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import ModelCheckpoint
# define KFOLD-fold cross validation test harness
kfold = StratifiedKFold(n_splits = KFOLD, shuffle = True, random_state = SEED)
cvscores = []
ii = 1
for train, test in kfold.split(x_subject, y_subject[:, 0]):
    print('Run ' + str(ii) + '...')
    # create callbacks
    model_name_str = 'GRAZ_MLP_' + \
                     '_run_' + str(ii)
    
    checkpoint = ModelCheckpoint(model_name_str, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    # Fit the model
    # initialize and create the model
    model = MLP_model(x_subject.shape[1:], dropout = DROPOUT, print_summary = False)
    
    # fit model. If you specify monitor=True, then the model will create callbacks
    # and write its state to a HDF5 file
    model.fit(x_subject[train], y_subject[train],
              epochs = NumbItr, 
              batch_size = 256, 
              verbose = 1, 
              validation_split = 0.1, callbacks = callbacks_list)

    # evaluate the model
    print('Evaluating model on test set...')
    scores = model.evaluate(x_subject[test], y_subject[test], verbose = 0)
    print("Result on test set: %s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
    cvscores.append(scores[1] * 100)
    ii += 1
    
# print some evaluation statistics and write results to file
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
cv_all_subjects = np.asarray(cvscores)
print('Saving CV values to file....')
np.savetxt('GRAZ_CV_' + 'MLP_' + str(DROPOUT) + 'do'+'.csv', 
            cv_all_subjects, delimiter = ',', fmt = '%2.4f')
print('CV values successfully saved!\n')

Run 1...
Train on 7231 samples, validate on 804 samples
Epoch 1/10
7231/7231 [==============================] - 6s 770us/step - loss: 0.6598 - acc: 0.6581 - val_loss: 0.8347 - val_acc: 0.4552

## Load the trained model 

In [14]:
model.save('MLPmonitoring.h5')  # creates a HDF5 file 'my_model.h5'
model2 = load_model('MLPmonitoring.h5', 
                                 custom_objects={'Spectrogram': kapre.time_frequency.Spectrogram, 
                                                 'Normalization2D': kapre.utils.Normalization2D})

int_axis=1 passed but is ignored, str_axis is used instead.
